In [29]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, RNN, GRUCell, Dense, TimeDistributed, AdditiveAttention, Concatenate
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [30]:
df = pd.read_csv("CNN_Articles.csv", index_col=0)


In [31]:
df = df[['text','headline']].dropna()
df.rename(columns={'text':'article'}, inplace=True)

def clean_text_keep_symbols(text):
    text = str(text).lower()
    text = text.replace('\n',' ')
    text = ' '.join(text.split())
    return text

df['article'] = df['article'].apply(clean_text_keep_symbols)
df['headline'] = df['headline'].apply(clean_text_keep_symbols)

In [32]:
MAX_ARTICLE_LEN = 400
MAX_HEADLINE_LEN = 20

train_df = df.sample(frac=0.8, random_state=42)
val_df = df.drop(train_df.index)

article_tokenizer = Tokenizer(oov_token="<OOV>")
article_tokenizer.fit_on_texts(train_df['article'])
headline_tokenizer = Tokenizer(oov_token="<OOV>")
headline_tokenizer.fit_on_texts(train_df['headline'])

In [33]:
X_train = pad_sequences(article_tokenizer.texts_to_sequences(train_df['article']),
                        maxlen=MAX_ARTICLE_LEN, padding='post')
X_val = pad_sequences(article_tokenizer.texts_to_sequences(val_df['article']),
                      maxlen=MAX_ARTICLE_LEN, padding='post')

y_train = pad_sequences(headline_tokenizer.texts_to_sequences(train_df['headline']),
                        maxlen=MAX_HEADLINE_LEN, padding='post')
y_val = pad_sequences(headline_tokenizer.texts_to_sequences(val_df['headline']),
                      maxlen=MAX_HEADLINE_LEN, padding='post')

In [34]:
y_train_in = np.zeros_like(y_train)
y_train_in[:,1:] = y_train[:,:-1]
y_train_in[:,0] = headline_tokenizer.word_index['<OOV>']

y_val_in = np.zeros_like(y_val)
y_val_in[:,1:] = y_val[:,:-1]
y_val_in[:,0] = headline_tokenizer.word_index['<OOV>']

In [35]:
embedding_dim = 256
hidden_units = 256
article_vocab_size = len(article_tokenizer.word_index)+1
headline_vocab_size = len(headline_tokenizer.word_index)+1

In [36]:
encoder_inputs = Input(shape=(MAX_ARTICLE_LEN,))
enc_emb = Embedding(article_vocab_size, embedding_dim)(encoder_inputs)

encoder_rnn = RNN(GRUCell(hidden_units), return_sequences=True, return_state=True)
encoder_outputs, state_h = encoder_rnn(enc_emb)

In [37]:
decoder_inputs = Input(shape=(MAX_HEADLINE_LEN,))
dec_emb = Embedding(headline_vocab_size, embedding_dim)(decoder_inputs)

decoder_rnn = RNN(GRUCell(hidden_units), return_sequences=True, return_state=True)
decoder_outputs, _ = decoder_rnn(dec_emb, initial_state=state_h)

In [38]:
attention = AdditiveAttention()([decoder_outputs, encoder_outputs])
decoder_combined_context = Concatenate()([decoder_outputs, attention])

In [39]:
outputs = TimeDistributed(Dense(headline_vocab_size, activation='softmax'))(decoder_combined_context)


In [40]:
model = Model([encoder_inputs, decoder_inputs], outputs)
model.compile(optimizer=Adam(1e-3),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2       │ (None, 400)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_3       │ (None, 20)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_2         │ (None, 400, 256)  │ 20,183,808 │ input_layer_2[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_3         │ (None, 20, 256)   │  2,612,992 │ input_layer_3[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rnn_2 (RNN)         │ [(None, 400,      │    394,752 │ embedding_2[0][0] │
│                     │ 256), (None,      │            │                   │
│                     │ 256)]             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rnn_3 (RNN)         │ [(None, 20, 256), │    394,752 │ embedding_3[0][0… │
│                     │ (None, 256)]      │            │ rnn_2[0][1]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ additive_attention… │ (None, 20, 256)   │        256 │ rnn_3[0][0],      │
│ (AdditiveAttention) │                   │            │ rnn_2[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_1       │ (None, 20, 512)   │          0 │ rnn_3[0][0],      │
│ (Concatenate)       │                   │            │ additive_attenti… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ time_distributed_1  │ (None, 20, 10207) │  5,236,191 │ concatenate_1[0]… │
│ (TimeDistributed)   │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 28,822,751 (109.95 MB)

 Trainable params: 28,822,751 (109.95 MB)

 Non-trainable params: 0 (0.00 B)

In [41]:
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True,verbose=1)


In [42]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
early_stop = EarlyStopping(
    monitor='val_loss',
    patience=3,
    restore_best_weights=True
)

checkpoint = ModelCheckpoint(
    filepath='headline_model_best.h5',
    monitor='val_loss',
    save_best_only=True,
    verbose=1
)
history = model.fit(
    [X_train, y_train_in],
    y_train[..., np.newaxis],
    validation_data=([X_val, y_val_in], y_val[..., np.newaxis]),
    epochs=10,
    batch_size=64,
    callbacks=[early_stop, checkpoint]
)

Epoch 1/10
117/117 ━━━━━━━━━━━━━━━━━━━━ 0s 172ms/step - accuracy: 0.3416 - loss: 6.2688
Epoch 1: val_loss improved from inf to 4.89961, saving model to headline_model_best.h5


117/117 ━━━━━━━━━━━━━━━━━━━━ 37s 234ms/step - accuracy: 0.3418 - loss: 6.2615 - val_accuracy: 0.3933 - val_loss: 4.8996
Epoch 2/10
117/117 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step - accuracy: 0.3924 - loss: 4.7716
Epoch 2: val_loss improved from 4.89961 to 4.75575, saving model to headline_model_best.h5


117/117 ━━━━━━━━━━━━━━━━━━━━ 30s 184ms/step - accuracy: 0.3924 - loss: 4.7715 - val_accuracy: 0.3957 - val_loss: 4.7557
Epoch 3/10
117/117 ━━━━━━━━━━━━━━━━━━━━ 0s 119ms/step - accuracy: 0.4039 - loss: 4.5323
Epoch 3: val_loss improved from 4.75575 to 4.50288, saving model to headline_model_best.h5


117/117 ━━━━━━━━━━━━━━━━━━━━ 40s 176ms/step - accuracy: 0.4039 - loss: 4.5322 - val_accuracy: 0.4059 - val_loss: 4.5029
Epoch 4/10
117/117 ━━━━━━━━━━━━━━━━━━━━ 0s 118ms/step - accuracy: 0.4128 - loss: 4.1720
Epoch 4: val_loss improved from 4.50288 to 4.19236, saving model to headline_model_best.h5


117/117 ━━━━━━━━━━━━━━━━━━━━ 37s 141ms/step - accuracy: 0.4129 - loss: 4.1715 - val_accuracy: 0.4181 - val_loss: 4.1924
Epoch 5/10
117/117 ━━━━━━━━━━━━━━━━━━━━ 0s 119ms/step - accuracy: 0.4328 - loss: 3.6939
Epoch 5: val_loss improved from 4.19236 to 3.85704, saving model to headline_model_best.h5


...

117/117 ━━━━━━━━━━━━━━━━━━━━ 21s 138ms/step - accuracy: 0.4720 - loss: 3.1754 - val_accuracy: 0.4696 - val_loss: 3.4950
Epoch 7/10
117/117 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step - accuracy: 0.5330 - loss: 2.6466
Epoch 7: val_loss improved from 3.49503 to 3.15371, saving model to headline_model_best.h5


117/117 ━━━━━━━━━━━━━━━━━━━━ 20s 135ms/step - accuracy: 0.5330 - loss: 2.6463 - val_accuracy: 0.5080 - val_loss: 3.1537
Epoch 8/10
117/117 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step - accuracy: 0.6067 - loss: 2.1726
Epoch 8: val_loss improved from 3.15371 to 2.84016, saving model to headline_model_best.h5


117/117 ━━━━━━━━━━━━━━━━━━━━ 21s 136ms/step - accuracy: 0.6067 - loss: 2.1723 - val_accuracy: 0.5526 - val_loss: 2.8402
Epoch 9/10
117/117 ━━━━━━━━━━━━━━━━━━━━ 0s 118ms/step - accuracy: 0.6775 - loss: 1.7544
Epoch 9: val_loss improved from 2.84016 to 2.55938, saving model to headline_model_best.h5


117/117 ━━━━━━━━━━━━━━━━━━━━ 21s 137ms/step - accuracy: 0.6775 - loss: 1.7542 - val_accuracy: 0.6003 - val_loss: 2.5594
Epoch 10/10
117/117 ━━━━━━━━━━━━━━━━━━━━ 0s 118ms/step - accuracy: 0.7433 - loss: 1.3988
Epoch 10: val_loss improved from 2.55938 to 2.31198, saving model to headline_model_best.h5


117/117 ━━━━━━━━━━━━━━━━━━━━ 20s 136ms/step - accuracy: 0.7434 - loss: 1.3987 - val_accuracy: 0.6419 - val_loss: 2.3120


In [43]:
import pickle
import os

os.makedirs("models", exist_ok=True)

model.save("models/headline_model_best.h5")

with open("models/article_tokenizer.pkl", "wb") as f:
    pickle.dump(article_tokenizer, f, protocol=pickle.HIGHEST_PROTOCOL)

with open("models/headline_tokenizer.pkl", "wb") as f:
    pickle.dump(headline_tokenizer, f, protocol=pickle.HIGHEST_PROTOCOL)

print("✅ Model and tokenizers saved successfully!")


✅ Model and tokenizers saved successfully!
